In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

device = torch.device("cuda") # GPU에서 실행하려면 이 주석을 제거하세요.


In [2]:
like_raw=pd.read_csv('likes.csv')

post=like_raw['post'].drop_duplicates().reset_index(drop=True)
account=like_raw['account'].drop_duplicates().reset_index(drop=True)
start = torch.cuda.Event(enable_timing=True) 
end = torch.cuda.Event(enable_timing=True) 



def loss_function(C,P,xTy,X,Y,r_lambda):
    predict_error=torch.square(P-xTy)
    confidence_error=torch.sum(C*predict_error)
    regularization=r_lambda*(torch.sum(torch.square(X))+torch.sum(torch.square(Y)))
    total_loss=confidence_error+regularization
    return torch.sum(predict_error),confidence_error,regularization,total_loss

def optimize_user(X,Y,C,P,nu,nf,r_lambda):
    yT=Y.transpose(0,1)
    for u in range(nu):
        Cu=torch.diag(C[u])
        yT_Cu_y=torch.matmul(torch.matmul(yT,Cu),Y)
        lI=torch.mul(r_lambda,torch.eye(nf)).to(device)
        yT_Cu_pu=torch.matmul(torch.matmul(yT,Cu),P[u])
        X[u]=torch.linalg.solve(yT_Cu_y+lI,yT_Cu_pu)
        
def optimize_item(X,Y,C,P,ni,nf,r_lambda):
    xT=X.transpose(0,1)
    for i in range(ni):
        Ci=torch.diag(C[:,i])
        xT_Ci_x=torch.matmul(torch.matmul(xT,Ci),X)
        lI=torch.mul(r_lambda,torch.eye(nf)).to(device)
        xT_Ci_pi=torch.matmul(torch.matmul(xT,Ci),P[:,i])
        Y[i]=torch.linalg.solve(xT_Ci_x+lI,xT_Ci_pi)
        

In [ ]:
like_n=pd.read_csv('like_p.csv').to_numpy()

like_t=torch.from_numpy(like_n)

dataset=TensorDataset(like_t)

dataloader=DataLoader(dataset,batch_size=3000)

epoch=15
predicts=[]

for i in range(epoch):

    for batch_idx,samples in enumerate(dataloader):
        
        r_lambda=40
        nf=200
        alpha=40
        
        nu=samples[0].shape[0]
        ni=samples[0].shape[1]


        X=torch.rand(nu,nf,device=device)*0.01
        Y=torch.rand(ni,nf,device=device)*0.01
        
        P=torch.tensor(samples[0],device=device)
        P[P>0]=1
        P=P.float()

        C=1.0+alpha*samples[0].cuda()


        predict_errors=[]
        confidence_errors=[]
        regularization_list=[]
        total_losses=[]
        

        
        if i!=0:
            
            optimize_user(X,Y,C,P,nu,nf,r_lambda)
            optimize_item(X,Y,C,P,ni,nf,r_lambda)
        
        predict=torch.matmul(X,Y.transpose(0,1))
        
        if i==(epoch-1):
            print(predicts)
            predicts.append(predict)
            print(predicts)
        
        predict_error,confidence_error,regularization,total_loss=loss_function(C,P,predict,X,Y,r_lambda)

        predict_errors.append(predict_error)
        confidence_errors.append(confidence_error)
        regularization_list.append(regularization)
        total_losses.append(total_loss)


        print('--------------------step %d--------------------------'%i)
        
        print('predict error: %f' %predict_error)
        print('confidence_error: %f' %confidence_error)
        print('regularization: %f'%regularization)
        print('total_loss: %f' %total_loss)
        
        
        


#predicts.append(torch.matmul(X,Y.transpose(0,1)))

predict=torch.cat(predicts,dim=0)

print('final predict')
print([predict])

print(predict[0])



/home/mojuk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


--------------------step 0--------------------------
predict error: 19944.119141
confidence_error: 816062.375000
regularization: 947.070374
total_loss: 817009.437500
--------------------step 0--------------------------
predict error: 6175.541504
confidence_error: 251996.734375
regularization: 724.001160
total_loss: 252720.734375
--------------------step 1--------------------------
predict error: 33252.273438
confidence_error: 319380.937500
regularization: 134520.343750
total_loss: 453901.281250
--------------------step 1--------------------------
predict error: 12205.439453
confidence_error: 142612.031250
regularization: 26150.107422
total_loss: 168762.140625
--------------------step 2--------------------------
predict error: 32973.472656
confidence_error: 319668.500000
regularization: 134137.281250
total_loss: 453805.781250
--------------------step 2--------------------------
predict error: 12133.195312
confidence_error: 141427.812500
regularization: 26651.494141
total_loss: 168079.31

In [14]:
predict_n=predict.cpu().numpy()
predict_d=pd.DataFrame(predict_n,index=account,columns=post)
predict_d.to_csv("predict.csv")


post,5f0c207664e2ef528e350cee,5f0c20b064e2ef528e350cfd,5f0c254e34765359af1215ae,5f047592bdb68f253ea0247a,5f046907b6cfc8253f73d805,5f045d6abdb68f253ea00b16,5f073c1a1cc750db80d5612a,5f0c27f134765359af1215d9,5f07167a1cc750db80d55d03,5f0744a9293361db7fd45f14,...,5ff64addc68ab9a449edaea0,5ff693a1526595c0d58670c8,5ff6b5a96639101a29b8b2a9,5ff8394ac4201aa5dce09e33,5ff843f898f1b83219c539dc,5ff6c7de12dd5ee63e034049,5ff95bf3885d5b0d610fff29,5ffbdcfe6d14769dd02dcacd,5ffc319c4d330b131782059f,5ffaa8e54a725404a82852e3
account,,,,,,,,,,,,,,,,,,,,,
5f0c228b34765359af121581,0.250329,0.191904,0.232428,0.252986,0.269019,0.276484,0.242313,0.324793,0.241424,0.199115,...,0.107737,0.252354,0.172256,0.182569,0.223158,0.076793,0.145717,0.190799,0.174547,0.042367
5edf9f8e031bf169c41c41a4,1.302167,1.212505,1.319488,1.518938,1.471571,1.529393,1.431746,1.388836,1.566651,1.206935,...,0.672566,1.421170,1.069109,1.198107,1.437323,0.481902,0.895688,1.168435,1.053898,0.267506
5ee2339f84ae69f7c7e624d9,0.995475,0.898475,1.017112,1.307739,1.244029,1.287741,1.274482,1.021580,1.323444,0.905818,...,0.482853,1.075358,0.777099,0.868752,1.049443,0.345344,0.645666,0.850527,0.757098,0.186571
5f0c23bce021eb59ae24f7b5,1.184107,1.124920,1.189947,1.470797,1.453343,1.495913,1.413504,1.297985,1.458501,1.125380,...,0.619536,1.336417,1.004051,1.140112,1.337325,0.446736,0.839555,1.071365,0.993310,0.245783
5ee2480577ba98467e2daf47,1.409082,1.360329,1.441603,1.784051,1.756625,1.786013,1.709173,1.546353,1.864274,1.368934,...,0.755456,1.664470,1.202641,1.333739,1.544052,0.544257,1.014241,1.338343,1.198806,0.301088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5fef3f3cbaad153c29701c53,0.011361,0.000000,0.011361,0.039495,0.037444,0.039053,0.034565,0.009680,0.018828,0.015367,...,0.000000,0.021347,0.011386,0.016021,0.020018,0.010271,0.002271,0.011734,0.009266,0.000000
5ffd44f5207cd5b5e4009cd4,0.011376,0.000000,0.011376,0.042468,0.038512,0.037249,0.038556,0.010337,0.034998,0.017468,...,0.000000,0.026321,0.012989,0.016185,0.013561,0.010408,0.001788,0.008373,0.009321,0.000000
5ffd469f207cd5b5e4009ce3,0.022228,0.000000,0.022228,0.081203,0.074565,0.074340,0.072085,0.019588,0.130500,0.041788,...,0.000000,0.041575,0.026022,0.031174,0.035288,0.018888,0.003807,0.015976,0.019445,0.000000
